# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) are ignored in the calculations.

In the first cell, replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):

```python
key=YOUR-LASTFM-API-KEY
username=YOUR-LASTFM-USERNAME
```

In [2]:
import requests, json, time, pandas as pd

key='03dd24d9c70691e2ae5210b0986c2397'
username='jeepster71boy'

In [3]:
# how long to pause between consecutive API requests
pause_duration = 0.2

## First get your all-time most played tracks, artists, and albums

In [4]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 0 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [5]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('data/lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,Miranda Lambert,For the Birds,47
1,Miranda Lambert,We Should Be Friends,43
2,Red Hot Chili Peppers,Soul to Squeeze,39
3,Miranda Lambert,Well-Rested,38
4,Miranda Lambert,To Learn Her,36


In [6]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('data/lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,Miranda Lambert,1234
1,Explosions in the Sky,670
2,Dean Martin,530
3,Norah Jones,423
4,Jack Johnson,346


In [7]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('data/lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,Miranda Lambert,The Weight of These Wings,693
1,Miranda Lambert,Platinum,289
2,Dean Martin,Dino: The Essential Dean Martin,275
3,Explosions in the Sky,The Wilderness,140
4,Explosions in the Sky,Manglehorn (Original Motion Picture Soundtrack),138


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [8]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = int(response[method]['@attr']['totalPages'])
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print('{} total pages to retrieve'.format(total_pages))
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print(page, end=' ')
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [9]:
# get all scrobbled tracks ever, in order of recency (pages=0 to get all)
scrobbles = get_scrobbles(pages=0)

99 total pages to retrieve
10 20 30 40 50 60 70 80 90 

In [10]:
# save the dataset
scrobbles.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print('{:,} total rows'.format(len(scrobbles)))
scrobbles.head()

19,632 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,Michael Bublé,611700cf-27f0-4dc9-ae80-c513a767853e,It's Time,cb15f5d0-1e03-48bb-b098-06f87e61494d,Save the Last Dance for Me,aeb8b330-e24c-463a-ad50-56dc7cb6ab36,1533327597,2018-08-03 20:19:57
1,Ingrid Michaelson,1fc494a1-9109-4081-a455-2d05bea9d2bf,Be OK,bd049998-44f1-4ca1-9b38-853b25772a12,Keep Breathing,74f11f87-85fc-4364-b77c-a638ecbb2319,1533327390,2018-08-03 20:16:30
2,Jon Peter Lewis,1804cd3a-4ca8-4fbb-be74-308dc8b3ebb6,Break The Silence,093f1aae-4c2b-4fd9-ba92-cb811d6a35ad,Tonight,2d3ce546-a3b6-4662-a68a-24d02abdebae,1533327256,2018-08-03 20:14:16
3,SafetySuit,9247edb7-1bb5-43f2-a58f-185a8ba6044f,Hallelujah,177da40b-4249-4537-a614-47e392e8f57e,Never Stop (Wedding Version),6929e8cf-5379-4b6f-b77b-bdaadc6ed675,1533327081,2018-08-03 20:11:21
4,Howard Shore,9b58672a-e68e-4972-956e-a8985a165a1f,"Ed Sheeran, The Hobbit: The Desolation Of Smau...",,The Hobbit: The Desolation Of Smaug (Original ...,,1533326778,2018-08-03 20:06:18
